In [ ]:
import os
import math

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm, transforms
from mpl_axes_aligner import align
import numpy as np
np.seterr(divide='ignore')
from scipy.stats import poisson, norm
from scipy import optimize as opti
import pandas as pd
from tqdm import tqdm
from scipy import special
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.signal import savgol_filter
import h5py
import torch
from torchviz import make_dot

import wf_func as wff

In [ ]:
def normcombine(x, m, s, a):
    return a[0] * norm.pdf((x - m[0]) / s[0]) + a[1] * norm.pdf((x - m[1]) / s[1])

def normcombine2d(x, m, s, a, rho):
    return a[0, 0] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 0]], cov=matrix(s[0, 0], s[1, 0], rho[0, 0])) + a[0, 1] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 1]], cov=matrix(s[0, 0], s[1, 1], rho[0, 1])) + a[1, 0] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 0]], cov=matrix(s[0, 1], s[1, 0], rho[1, 0])) + a[1, 1] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 1]], cov=matrix(s[0, 1], s[1, 1], rho[1, 1]))

def matrix(sx, sy, rho):
    return np.array([[sx ** 2, rho * sx * sy], [rho * sx * sy, sy ** 2]])

def chargehist(t):
    c = norm.pdf(t, loc=160., scale=40.)
#     q1 = 150.8
#     sigma = 37.59
#     w = 2.433e-5
#     alpha = 0.01335
#     mu = 2.851e-5
#     c = np.exp(-mu)*(w*alpha*np.exp(-alpha*t))
#     c = c + mu*np.exp(-mu)*(
#         (1-w)/(sigma*np.sqrt(2*np.pi))*np.exp(-(t-q1)**2/(2*sigma**2))+
#         w*(alpha/2*np.exp(-alpha*(t-q1-alpha/2*sigma**2))*(1+special.erf(t-q1-alpha*sigma**2)/(np.sqrt(2)*sigma))))
    return c

In [ ]:
Thres = {'xiaopeip':0.2, 'lucyddm':0.2, 'fftrans':0.1, 'findpeak':0.1, 'threshold':0.1, 'omp':0}
std = 1.
spe_pre = wff.read_model('spe.h5', 1)
window = 1029
pan = np.arange(window)
p = spe_pre[0]['parameters']
t = np.arange(window).astype(np.float)
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
t = np.arange(0, 100, 0.1)
ax.plot(t, wff.spe(t, p[0], p[1], p[2]), color='b', label='Single PE response')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.grid()
ax.set_xlim(0, 80)
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax.legend()
fig.savefig('Note/figures/spe.pgf')
fig.savefig('Note/figures/spe.pdf')
plt.close()

In [ ]:
fig = plt.figure(figsize=(8, 6))
t = np.arange(-4 * 5, 5 * 20, 0.1)
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.95, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(t, wff.convolve_exp_norm(t, 20, 0), label=r'$(20,0)$', color='g')
ax.plot(t, wff.convolve_exp_norm(t, 0, 5), label=r'$(0,5)$', color='r')
ax.plot(t, wff.convolve_exp_norm(t, 20, 5), label=r'$(20,5)$', color='b')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.grid()
ax.set_xlim(xmin=-4 * int(5))
ax.set_ylabel(r'$\mathrm{PDF}$')
ax.legend(title=r'$(\tau, \sigma)/\si{ns}$', loc='upper right')
ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
# ax.annotate(r'$t_{0}$', xy=(0, 0), xytext=(5, 0.01), arrowprops=dict(facecolor='k', shrink=0.1, width=0.1, headwidth=2))
fig.savefig('Note/figures/profile.pgf')
fig.savefig('Note/figures/profile.pdf')
plt.close()

In [ ]:
Mu = 4
Tau = 20
Sigma = 5
file = '4.0-20-5'

In [ ]:
with h5py.File('waveform/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    ent = ipt['Readout/Waveform'][:]
    tru = ipt['SimTriggerInfo/PEList'][:]
    gmu = ipt['SimTriggerInfo/PEList'].attrs['gmu']
    gsigma = ipt['SimTriggerInfo/PEList'].attrs['gsigma']
    t0truth = ipt['SimTruth/T'][:]

In [ ]:
t = np.array([0, 1, 2, 3])
a = np.array([0.5, 0.5, 0.5, 1])
b1 = np.array([0.5, 0.5, 1, 0.5])
b2 = np.array([1, 0.5, 0.5, 0.5])
fig = plt.figure(figsize=(15, 4))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 3, figure=fig, left=0.07, right=0.97, top=0.97, bottom=0.2, wspace=0.3, hspace=0.3)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[0, 2])
ax0.bar(t, a, color='r')
ax0.set_xlabel(r'$a$')
ax0.set_ylabel(r'$\mathrm{weight}\ a$')
ax0.set_xticks(t)
ax0.set_ylim(0, ax0.get_ylim()[1] * 1.05)
ax1.bar(t, b1, color='b')
ax1.set_xlabel(r'$b_{1}$')
ax1.set_ylabel(r'$\mathrm{weight}\ b_{1}$')
ax1.set_xticks(t)
ax1.set_ylim(0, ax1.get_ylim()[1] * 1.05)
ax2.bar(t, b2, color='b')
ax2.set_xlabel(r'$b_{2}$')
ax2.set_ylabel(r'$\mathrm{weight}\ b_{2}$')
ax2.set_xticks(t)
ax2.set_ylim(0, ax2.get_ylim()[1] * 1.05)
fig.savefig('Note/figures/tab.pgf')
fig.savefig('Note/figures/tab.pdf')
fig.clf()
plt.close(fig)

In [ ]:
i = 2
cid = ent[i]['ChannelID']
eid = ent[i]['TriggerNo']
truth = np.sort(tru[(tru['TriggerNo'] == eid) & (tru['PMTId'] == cid)], kind='stable', order=['TriggerNo', 'PMTId', 'HitPosInWindow'])
wave = ent[i]['Waveform'].astype(np.float) * spe_pre[ent[i]['ChannelID']]['epulse']
df = pd.DataFrame(truth)
df = df.rename(columns={'HitPosInWindow':'HitTime'})
charge = df['Charge'].copy()
hittime = df['HitTime'].copy()
df = df.astype({'Charge': 'float32'})
df = df.astype({'TriggerNo' : 'str', 'PMTId' : 'str', 'HitTime' : 'str', 'Charge': 'str'})
df['HitTime'] = ['{:.02f}'.format(s) for s in hittime]
df['Charge'] = ['{:.02f}'.format(s) for s in charge]
df

In [ ]:
ind = np.argwhere(wave > spe_pre[cid]['std'] * 5).flatten()
xmin = ((ind.min() - spe_pre[cid]['mar_l']) // 20 - 1) * 20
xmax = max(((ind.max() + spe_pre[cid]['mar_r']) // 20 + 1) * 20, xmin + 200)

In [ ]:
mu0 = np.arange(1, int(Mu + 5 * np.sqrt(Mu)))
n_t = np.arange(1, 100)
p_t = special.comb(mu0, 2)[:, None] * np.power(wff.convolve_exp_norm(np.arange(1029) - 200, Tau, Sigma) / n_t[:, None], 2).sum(axis=1)
n0 = np.array([n_t[p_t[i] < max(1e-2, np.sort(p_t[i])[1])].min() for i in range(len(mu0))])
ndict = dict(zip(mu0, n0))
b_t0 = [0., 600.]
p = spe_pre[cid]['parameters']
nsp = 4
nstd = 3
D = 500
intn = wave.sum() / gmu
mu_t = abs(wave.sum() / gmu)
n = ndict[min(math.ceil(mu_t + 3 * np.sqrt(mu_t)), max(mu0))]
factor = np.linalg.norm(spe_pre[cid]['spe'])
A, wave_r, tlist, t0_t, t0_init_delta, char_init, left_wave, right_wave = wff.initial_params(wave, spe_pre[cid], Mu, Tau, Sigma, gmu, 0.2, p, nsp, nstd, is_t0=True, is_delta=False, n=n, nshannon=1)
A = A / factor
la = np.clip(mu_t * wff.convolve_exp_norm(tlist - t0_t, Tau, Sigma) / n, 1e-8, 1 - 1e-8)
def optit0mu(t0, mu, n, xmmse_star, psy_star, la, factor):
    ys = np.log(psy_star) - np.where(xmmse_star != 0, np.log(la), np.log(1 - la)).sum(axis=1)
    ys = np.exp(ys - ys.max()) / np.sum(np.exp(ys - ys.max()))
    btlist = np.arange(t0 - 3 * Sigma, t0 + 3 * Sigma + 1e-6, 0.2)
    mulist = np.arange(max(0.2, mu - 3 * np.sqrt(mu)), mu + 3 * np.sqrt(mu), 0.2)
    b_mu = [max(0.2, mu - 5 * np.sqrt(mu)), mu + 5 * np.sqrt(mu)]
    tlist_pan = np.sort(np.unique(np.hstack(np.arange(0, window)[:, None] + np.arange(0, 1, 1 / n))))
    As = np.zeros((len(xmmse_star), len(tlist_pan)))
    As[:, np.isin(tlist_pan, tlist)] = np.clip(xmmse_star, 0, np.inf) / factor
    assert sum(np.sum(As, axis=0) > 0) > 0

    # optimize t0
    logL = lambda t0 : -1 * np.sum(special.logsumexp((np.log(np.clip(wff.convolve_exp_norm(tlist_pan - t0, Tau, Sigma), np.finfo(np.float64).tiny, np.inf))[None, :] * np.where(As > 0, 1, 0)).sum(axis=1), b=ys))
    logLv_btlist = np.vectorize(logL)(btlist)
    t0 = opti.fmin_l_bfgs_b(logL, x0=[btlist[np.argmin(logLv_btlist)]], approx_grad=True, bounds=[b_t0], maxfun=50000)[0]

    # optimize mu
    def likelihood(mu):
        a = np.clip(mu * wff.convolve_exp_norm(tlist_pan - t0, Tau, Sigma) / n, np.finfo(np.float64).eps, 1) # use tlist_pan not tlist
        li = -special.logsumexp(np.where(As > 0, np.log(a), np.log(1 - a)).sum(axis=1), b=ys)
        return li
    like = np.array([likelihood(mulist[j]) for j in range(len(mulist))])
    mu = opti.fmin_l_bfgs_b(likelihood, x0=mulist[like.argmin()], approx_grad=True, bounds=[b_mu], maxfun=50000)[0]
    return t0, mu, ys
xmmse_star, psy_star, nu_star, T_star, d_tot_i, d_max_i = wff.fbmpr_fxn_reduced(wave_r, A, la, spe_pre[cid]['std'] ** 2, (gsigma * factor / gmu) ** 2, factor, D, stop=0)
t0, mu, ys = optit0mu(t0_t, mu_t, n, xmmse_star, psy_star, la, factor)
while abs(t0_t - t0) > 1e-3:
    t0_t = t0
    mu_t = mu
    t0, mu, ys = optit0mu(t0_t, mu_t, n, xmmse_star, psy_star, la, factor)
xmmse_most = xmmse_star[0]
pet = tlist[xmmse_most > 0]
cha = xmmse_most[xmmse_most > 0] / factor
pet, pwe = wff.clip(pet, cha, 0.0)
pwe = pwe
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/fbmp.pgf')
fig.savefig('Note/figures/fbmp.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
print((t0 - t0truth[i]['T0']).item())

In [ ]:
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'].min(), 0, wave.sum() / gmu, color='r', label='Record')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
rise = truth['HitPosInWindow'].min()
vsum = wave.sum()
ax2.annotate(fr'${rise:.{4}}\mathrm{{ns}}, {vsum:.{4}}\mathrm{{mV}}\cdot\mathrm{{ns}}$', xy=(truth['HitPosInWindow'].min(), wave.sum()*2/3), xytext=(truth['HitPosInWindow'].min()+120, wave.sum()*1/2), arrowprops=dict(facecolor='k', shrink=0.1, width=0.1, headwidth=2))
ax.set_ylim(-5, ax.get_ylim()[1] * 1.05)
ax2.set_ylim(-5 / gmu, ax2.get_ylim()[1] * 1.05)
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/previous.pgf')
fig.savefig('Note/figures/previous.pdf')
fig.clf()
plt.close(fig)

In [ ]:
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(wave, label='Waveform')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax.set_xlim(0, len(wave))
ax.set_ylim(-5, ax.get_ylim()[1] * 1.05)
ax.legend()
fig.savefig('Note/figures/wave.pgf')
fig.savefig('Note/figures/wave.pdf')
fig.clf()
plt.close(fig)

fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.vlines(truth['HitPosInWindow'], 0, truth['Charge'] / gmu, color='r', label='Charge')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Charge}$')
ax.set_xlim(0, len(wave))
ax.set_ylim(bottom=0)
ax.legend()
fig.savefig('Note/figures/charge.pgf')
fig.savefig('Note/figures/charge.pdf')
fig.clf()
plt.close(fig)

In [ ]:
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'], 0, truth['Charge'] / gmu, color='r', label='Charge')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax.set_ylim(bottom=-5)
ax2.set_ylim(bottom=-5 / gmu)
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/goal.pgf')
fig.savefig('Note/figures/goal.pdf')
fig.clf()
plt.close(fig)

In [ ]:
print(wave.sum())
print(truth['Charge'][truth['Charge'] > 0].sum()) # made by noise

In [ ]:
t = np.load('result/takara/char/Channel00/testing_record_2021-04-07_07:22:22.npz')['arr_0']
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(np.arange(1, len(t)+1), t, label='W-dist', color='C1')
ax.set_xlabel(r'$\mathrm{epoch}$')
ax.set_ylabel(r'$\mathrm{Wasserstein\ Distance}/\si{ns}$')
ax.legend()
ax.grid()
fig.savefig('Note/figures/epoch.pgf')
fig.savefig('Note/figures/epoch.pdf')
fig.clf()
plt.close(fig)

In [ ]:
pet, pwe = wff.threshold(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['threshold'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
ax2.set_ylim(top=pwe.max()*1.2)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/threshold.pgf')
fig.savefig('Note/figures/threshold.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

In [ ]:
pet, pwe = wff.findpeak(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['findpeak'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=1.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
loc = pet + spe_pre[cid]['peak_c']
loc = loc[loc < window]
amp = wave[loc]
for j in range(len(loc)):
    ax.annotate('', xy=(loc[j], amp[j]+5), xytext=(loc[j], amp[j]+15), arrowprops=dict(facecolor='k', shrink=0.01, width=0.5, headwidth=2))
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
ax2.set_ylim(top=pwe.max()*1.2)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/findpeak.pgf')
fig.savefig('Note/figures/findpeak.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

In [ ]:
pet, pwe = wff.waveformfft(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['fftrans'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/fftrans.pgf')
fig.savefig('Note/figures/fftrans.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

In [ ]:
pet, pwe = wff.lucyddm(wave, spe_pre[cid]['spe'])
pet, pwe = wff.clip(pet, pwe, Thres['lucyddm'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/lucyddm.pgf')
fig.savefig('Note/figures/lucyddm.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

In [ ]:
with h5py.File('result/takara/char/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    photoelec = ipt['photoelectron'][:]
s = photoelec[(photoelec['TriggerNo'] == eid) & (photoelec['ChannelID'] == cid)]
pet = s['HitPosInWindow']
pwe = s['Charge']
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/takara.pgf')
fig.savefig('Note/figures/takara.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

In [ ]:
with h5py.File('result/mcmc/char/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    photoelec = ipt['photoelectron'][:]
s = photoelec[(photoelec['TriggerNo'] == eid) & (photoelec['ChannelID'] == cid)]
pet = s['HitPosInWindow']
pwe = s['Charge']
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/mcmc.pgf')
fig.savefig('Note/figures/mcmc.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

In [ ]:
pet, pwe = wff.xiaopeip(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['xiaopeip'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/xiaopeip.pgf')
fig.savefig('Note/figures/xiaopeip.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p, fold='/tmp')
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

In [ ]:
with h5py.File('result/mcmc/solu/' + file + '.h5', 'r', libver='latest', swmr=True) as soluf, h5py.File('waveform/' + file + '.h5', 'r', libver='latest', swmr=True) as wavef:
    start = wavef['SimTruth/T'][:]
    time = soluf['starttime'][:]
data = time['tswave'] - start['T0']
vali = np.abs(data - np.mean(data)) < 5 * np.std(data, ddof=-1)
data = data[vali]
fig = plt.figure(figsize=(8, 6))
ax = fig.add_axes((.1, .45, .85, .45))
ax.hist(data, bins=100, density=1)
ax.set_ylabel('Normalized Count')
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlim(data.min() - 0.05, data.max() + 0.05)
ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
axb = fig.add_axes((.1, .15, .85, .3))
axb.boxplot(data, vert=False, sym='', patch_artist=True)
axb.set_xlabel(r'$\Delta t_{0}/\si{ns}$')
axb.set_yticks([])
axb.set_xlim(ax.get_xlim())
fig.savefig('Note/figures/mcmct0hist.pgf')
fig.savefig('Note/figures/mcmct0hist.pdf')
plt.close(fig)

In [ ]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fbmp', 'mcmc']

for m in methods:
    with h5py.File('result/' + m + '/dist/' + file + '.h5', 'r', libver='latest', swmr=True) as distfile:
        dt = distfile['Record'][:]
    N = np.percentile(dt['wdist'], 95)
    M = 500

    penum = np.unique(dt['NPE'])
    l = min(50, penum.max())
    wdist_stats = np.full((l, 6), np.nan)
    edist_stats = np.full((l, 6), np.nan)
    for i in range(l):
        vali = dt['NPE'] == i+1
        if np.sum(vali) == 0:
            continue
        dtwpi = dt['wdist'][vali]
        dtepi = dt['RSS'][vali]
        wdist_stats[i, 0] = np.median(dtwpi)
        wdist_stats[i, 1] = np.median(np.abs(dtwpi - np.median(dtwpi)))
        wdist_stats[i, 2] = np.mean(dtwpi)
        wdist_stats[i, 3] = np.std(dtwpi)
        wdist_stats[i, 4] = np.percentile(dtwpi, 5)
        wdist_stats[i, 5] = np.percentile(dtwpi, 95)
        edist_stats[i, 0] = np.median(dtepi)
        edist_stats[i, 1] = np.median(np.abs(dtepi - np.median(dtepi)))
        edist_stats[i, 2] = np.mean(dtepi)
        edist_stats[i, 3] = np.std(dtepi)
        edist_stats[i, 4] = np.percentile(dtepi, 5)
        edist_stats[i, 5] = np.percentile(dtepi, 95)

    L = len(dt)
    data = dt['wdist']
    vali = np.abs(data - np.mean(data)) < 7 * np.std(data, ddof=-1)
    data = data[vali]
    fig = plt.figure(figsize=(8, 6))
    ax1 = fig.add_axes((.1, .12, .55, .8))
#     ey = np.vstack([wdist_stats[:, 0]-wdist_stats[:, 4], wdist_stats[:, 5]-wdist_stats[:, 0]])
#     ax1.errorbar(np.arange(1, l+1), wdist_stats[:, 0], yerr=ey, label=r'$W-dist^{95\%}_{5\%}$')
    boxdict = ax1.boxplot(np.array([dt['wdist'][dt['NPE'] == i+1] for i in range(l)], dtype=np.object), sym='', patch_artist=True)
    ax1.plot(np.arange(1, l + 1), wdist_stats[:, 0], label='W-dist')
    ax1.set_xlim(0, l + 1)
#     ax1.set_ylim(min([boxdict['whiskers'][2 * i].get_xydata()[1, 1] for i in range(l)]) * 0.95, max([boxdict['whiskers'][2 * i + 1].get_xydata()[1, 1] for i in range(l)]) * 1.05)
    ax1.set_ylim(0, max([boxdict['whiskers'][2 * i + 1].get_xydata()[1, 1] for i in range(l)]) * 1.05)
    ax1.set_xlabel(r'$N_{pe}$')
    ax1.set_ylabel(r'$\mathrm{W-dist}/\si{ns}$')
    # ax.set_title('W-dist vs NPE stats')
    ax1.legend()
    ax2 = fig.add_axes((.65, .12, .33, .8))
    ax2.hist(data, bins=np.arange(0, data.max(), np.percentile(data, 98) / 40), density=1, orientation='horizontal')
    ax2.set_xlabel('Normalized Count')
    ax2.set_xlim(0, ax2.get_xlim()[1] * 1.05)
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_ylim(ax1.get_ylim())
    fig.savefig('Note/figures/' + m + 'chargestats.pgf')
    fig.savefig('Note/figures/' + m + 'chargestats.pdf')
    plt.close(fig)

In [ ]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fbmp']
colors = {'truth':'k', 'lucyddm':'y', 'xiaopeip':'c', 'takara':'C0', 'fbmp':'r'}
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)
t = np.arange(0, 1000, 0.1)
ax.plot(t, norm.pdf(t, loc=160., scale=40.) / (1 - norm.cdf(0, loc=160., scale=40.)), label='$\mathrm{ChargePDF}$', color=colors['truth'])
th = 160 * 5 * 1e-4
labels = {'truth':'\mathrm{Truth}', 'lucyddm':'\mathrm{LucyDDM}', 'xiaopeip':'\mathrm{Fitting}', 'takara':'\mathrm{CNN}', 'fbmp':'\mathrm{FBMP}', 'fbmpwave':'\mathrm{FBMP}'}
for m in methods:
    ch = h5py.File('result/' + m + '/char/' + file + '.h5', 'r', libver='latest', swmr=True)
    cha = ch['photoelectron']['Charge']
    ax.hist(cha[cha > th], bins=np.arange(th, 400, 5), label='$'+labels[m]+'$', histtype='step', density=True, color=colors[m], linewidth=2.)
ax.set_xlim(0, 350)
ax.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax.legend()
ax.set_xlabel('$\mathrm{Charge}/\si{mV\cdot ns}$')
ax.set_ylabel(r'$\mathrm{Normalized\ Count}$')
plt.savefig('Note/figures/recchargehist.png')
plt.savefig('Note/figures/recchargehist.pdf')
plt.savefig('Note/figures/recchargehist.pgf')
plt.show()